In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForwardRegressor(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_sizes: list[int] = [64, 64], dropout: float = 0.1):
        super().__init__()
        layers = []
        in_features = input_size

        for hidden in hidden_sizes:
            layers.append(nn.Linear(in_features, hidden))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_features = hidden

        layers.append(nn.Linear(in_features, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

In [ ]:
from blocksnet.machine_learning.strategy.torch.tensor.supervised.regression_strategy import TorchTensorRegressionStrategy
from sklearn.preprocessing import StandardScaler

strategy = TorchTensorRegressionStrategy(FeedForwardRegressor, scalers={'x': StandardScaler(), 'y':StandardScaler()})

In [8]:
import pandas as pd

pd.read_pickle('blocks.pickle')

,geometry,residential,business,recreation,industrial,transport,special,agriculture,land_use,share,...,count_prison,capacity_landfill,count_landfill,capacity_plant_nursery,count_plant_nursery,capacity_greenhouse_complex,count_greenhouse_complex,capacity_warehouse,count_warehouse,site_area
0,"POLYGON ((349424.859 6631180.891, 349424.751 6...",0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,LandUse.TRANSPORT,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032316
1,"POLYGON ((352083.617 6633950.146, 352240.448 6...",0.099000,0.0,0.079912,0.000000,0.401072,0.0,0.417018,LandUse.AGRICULTURE,0.417018,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137101.088247
2,"POLYGON ((346700.642 6618453.176, 346681.107 6...",1.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,LandUse.RESIDENTIAL,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95239.084434
3,"POLYGON ((347043.363 6618261.219, 347042.608 6...",0.729125,0.0,0.270875,0.000000,0.000000,0.0,0.000000,LandUse.RESIDENTIAL,0.729125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125919.330984
4,"POLYGON ((354879.039 6618859.116, 354845.405 6...",0.454375,0.0,0.000000,0.000000,0.144935,0.0,0.399984,LandUse.RESIDENTIAL,0.454375,...,0.0,0.0,0.0,30.0,1.0,0.0,0.0,0.0,0.0,169860.164494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9363,"POLYGON ((346635.461 6647492.048, 346473.426 6...",0.937805,0.0,0.000000,0.000000,0.062195,0.0,0.000000,LandUse.RESIDENTIAL,0.937805,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16802.384600
9364,"POLYGON ((346361.221 6647603.446, 346328.072 6...",0.125516,0.0,0.266734,0.000000,0.607750,0.0,0.000000,LandUse.TRANSPORT,0.607750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47637.956433
9365,"POLYGON ((344109.285 6649134.367, 344000.209 6...",0.000000,0.0,0.763388,0.000000,0.231754,0.0,0.000000,LandUse.RECREATION,0.763388,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64487.699525
9366,"POLYGON ((346323.488 6649497.386, 346199.573 6...",0.404385,0.0,0.000000,0.471607,0.124008,0.0,0.000000,LandUse.INDUSTRIAL,0.471607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,1.0,42988.399080


In [7]:
strategy.model_params

{}